In [1]:
#on_kaggle = True
on_kaggle = False

#TRAIN_PREDICT = 'predict'
TRAIN_PREDICT = 'train'

if not on_kaggle:
    import os
    os.environ["CUDA_VISIBLE_DEVICES"]="3"

SEED = 42

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import gc
import os
from scipy.stats import spearmanr
from math import floor, ceil
import random

from unidecode import unidecode
import re
#import gensim
#import string

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow_hub as hub

#fix bug with using CuDNNLSTM
#gpus = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpus[0], True)

#from transformers import *

np.set_printoptions(suppress=True)

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    #torch.manual_seed(seed)
    #torch.cuda.manual_seed_all(seed)
    #torch.backends.cudnn.deterministic = True
    tf.random.set_seed(seed)
seed_everything(SEED)

#### 1. Read data and tokenizer

Read tokenizer and data, as well as defining the maximum sequence length that will be used for the input to Bert (maximum is usually 512 tokens)

In [3]:
if on_kaggle:
    PATH = '../input/google-quest-challenge/'
    CKPT_LOAD_PATH = ['../input/google-quest-qa-labeling-checkpoints/fold0-epoch3.h5py']
else:##offline
    PATH = '../data/'
    CKPT_SAVE_PATH = '../checkpoint/use-v1/'
    #CKPT_LOAD_PATH = ['../checkpoint/bert-base-uncased/']

MAX_SEQUENCE_LENGTH = 512

df_train = pd.read_csv(PATH+'train.csv')
df_test = pd.read_csv(PATH+'test.csv')
df_sub = pd.read_csv(PATH+'sample_submission.csv')
print('train shape =', df_train.shape)
print('test shape =', df_test.shape)

output_categories = list(df_train.columns[11:])
input_categories = list(df_train.columns[[1,2,5,9,10]])#9:category, 10:host
print('\noutput categories:\n\t', output_categories)
print('\ninput categories:\n\t', input_categories)

train shape = (6079, 41)
test shape = (476, 11)

output categories:
	 ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 'answer_type_reason_explanation', 'answer_well_written']

input categories:
	 ['question_title', 'question_body', 'answer', 'category', 'host']


In [19]:
#read USE sentence embedding model
#embeddings.shape==[1, 512]
if on_kaggle:
    pass
else:
    os.environ["TFHUB_CACHE_DIR"]="../data/"
    #use_embed = hub.load("https://hub.tensorflow.google.cn/google/universal-sentence-encoder-large/5")
    use_embed = hub.KerasLayer(handle="https://hub.tensorflow.google.cn/google/universal-sentence-encoder-large/5", 
                               trainable=True)

In [5]:
#%%time
import spacy
import en_core_web_sm

nlp = en_core_web_sm.load() #nlp = spacy.load('en_core_web_sm')

# qss, ass = [], []
# for i,row in tqdm(df_train.iterrows()):
#     if i not in list(range(100,200)):
#         continue
    
#     #single sample
#     t,q,a = row['question_title'],row['question_body'],row['answer']
#     ##for question text
#     doc_q = nlp(t+' '+q)
#     sentences_q = [sent.string.strip() for sent in doc_q.sents]
#     #print('[question]number of sentences: ', len(sentences_q))
#     out = []
#     #use 8 passages
#     ns = len(sentences_q)//8 #8 passages, ns sentences each passage
#     if ns==0:
#         ns = 1
#     for j in range(8):
#         p = ' '.join(s for s in sentences_q[ns*j:ns*(j+1)])
#         if j==7:
#             p += ' '.join(s for s in sentences_q[ns*(j+1):])
#         #print(j, p)
#         #out.append(use_embed([p])[0])
#         out.append(p)
#     #out = np.array(out).T
#     qss.append(out)
#     ##for answer text
#     doc_a = nlp(a)
#     sentences_a = [sent.string.strip() for sent in doc_a.sents]
#     #print('[answer]number of sentences: ', len(sentences_a))
#     out = []
#     #use 8 passages
#     ns = len(sentences_a)//8 #8 passages, ns sentences each passage
#     if ns==0:
#         ns = 1
#     for j in range(8):
#         p = ' '.join(s for s in sentences_a[ns*j:ns*(j+1)])
#         if j==7:
#             p += ' '.join(s for s in sentences_a[ns*(j+1):])
#         #print(j, p)
#         #out.append(use_embed([p])[0])
#         out.append(p)
#     #out = np.array(out).T
#     ass.append(out)

#### 2. Preprocessing functions

These are some functions that will be used to preprocess the raw text data into useable Bert inputs.

In [6]:
##text cleaning 1
import html
for col in ['question_title', 'question_body', 'answer']:
    df_train[col] = df_train[col].apply(html.unescape)
    df_test[col] = df_test[col].apply(html.unescape)

In [21]:
# def slide_window_sentence(text):
#     """text: raw text full
    
#     return: list of string(sentence), ['sentence 1', ..., 'sentence N']
#     """
#     tokens = tokenizer.encode_plus(text, None, 
#                                add_special_tokens=True, max_length=None, 
#                                truncation_strategy='longest_first')
#     a = tokens['input_ids']
    
#     t_len = len(a)
#     s_len = 8 #sentence/window length
#     overlap = 3 #how many words overlap
#     num_sentence = 512 #max number of sentences, if padding or truncate
#     #print(t_len, s_len)
#     sentences = []
#     i = 0
#     while True:
#         if i==0:
#             #print(a[0:s_len])
#             sentence = tokenizer.decode(a[0:s_len], skip_special_tokens=True, clean_up_tokenization_spaces=True)
#         else:
#             #print(a[i*s_len-overlap*i:(i+1)*s_len-overlap*i])
#             sentence = tokenizer.decode(a[i*s_len-overlap*i:(i+1)*s_len-overlap*i], skip_special_tokens=True, clean_up_tokenization_spaces=True)
#         sentences.append(sentence)
#         i += 1
#         if i*s_len-overlap*i>t_len:
#             break
#     if len(sentences)<num_sentence:
#         to_pad = ['']*(num_sentence-len(sentences))
#         sentences.extend(to_pad)
#     else:
#         sentences = sentences[:num_sentence]
#     #return sentences
#     return use_embed(sentences)

In [5]:
# idx = 1
# t,q,a = df_train.loc[idx, 'question_title'], df_train.loc[idx, 'question_body'], df_train.loc[idx, 'answer']
# print(t+' '+q+'\n'+'='*40+'\n'+a+'='*40)

# tokens = tokenizer.encode_plus(t+' '+q, None, 
#                                add_special_tokens=True, 
#                                max_length=None, 
#                                truncation_strategy='longest_first')
# print([tokenizer.ids_to_tokens[i] for i in tokens['input_ids']])
#print(' '.join(tokenizer.ids_to_tokens[i] for i in tokens['input_ids']))

# # #sentences = slide_window_sentence([2054]*6)
# # sentences = slide_window_sentence(tokens['input_ids'])
# # print(len(sentences))
# # sentences

# def slide_window(tokens, max_seq_len=384, doc_stride=128, cnt_sentence=16):
#     """
#     text: raw str, t+q+a
#     """
#     output = []
#     start_pos = 0
#     end_pos = -1
#     sentence_cnt = 0
#     all_tokens_len = len(tokens)
#     if start_pos<all_tokens_len:
#         if start_pos+max_seq_len>all_tokens_len:
#             seq = tokens[start_pos:]
            
    
#     return None

# print(len(tokens['input_ids']))
# slide_window(tokens['input_ids'], max_seq_len=384, doc_stride=128, cnt_sentence=16)


In [7]:
def count_special_char(x):
    special_character = re.compile(r'[^A-Za-z\.\-\?\!\,\#\@\% ]', re.IGNORECASE)
    x_ascii = unidecode(x)
    s = special_character.findall(x_ascii)
    #print(s)
    c = len(s)
    return c

def count_cap_char(x):
    c = sum(1 for l in x if l.isupper())
    return c

def count_unique_words(x):
    """returns a ratio"""
    special_character = re.compile(r'[^A-Za-z0-9]', re.IGNORECASE)
    a = [w for w in special_character.split(x) if w!='']
    r = len(set(a))/len(a)
    return r

#print(t)
#count_cap_char(t), len(t)
#count_special_char(t), len(t)
#string.printable

#from textblob import TextBlob
# print(t.lower())
# print('='*40)
# print(TextBlob(t.lower()).correct())

In [8]:
l = df_train.category.unique().tolist()
category2index = dict([(l[i],i+1) for i in range(len(l))])
category2index['UNK'] = 0

l = df_train.host.unique().tolist()
host2index = dict([(l[i],i+1) for i in range(len(l))])
host2index['UNK'] = 0

category2index

{'LIFE_ARTS': 1,
 'CULTURE': 2,
 'SCIENCE': 3,
 'STACKOVERFLOW': 4,
 'TECHNOLOGY': 5,
 'UNK': 0}

In [21]:
def compute_input_arrays(df, columns, tokenizer, max_sequence_length):
    special_char_q, special_char_a = [], []
    cap_char_q, cap_char_a = [], []
    unique_words_q, unique_words_a = [], []
    category_index, host_index = [], []
    qss, ass = [], []
    for row_id, instance in tqdm(df[columns].iterrows()):
        #if row_id>100:
        #    break
        t, q, a = instance.question_title, instance.question_body, instance.answer
        
        special_char_q.append(count_special_char(t+q)/len(t+q))
        special_char_a.append(count_special_char(a)/len(a))
        cap_char_q.append(count_cap_char(t+q)/len(t+q))
        cap_char_a.append(count_cap_char(a)/len(a))
        unique_words_q.append(count_unique_words(t+q))
        unique_words_a.append(count_unique_words(a))
        
        category, host = instance.category, instance.host
        category_index.append([category2index.get(category, 0)])
        host_index.append([host2index.get(host, 0)])
        
        ##USE model input "passages"
        ##for question text
        doc_q = nlp(t+' '+q)
        sentences_q = [sent.string.strip() for sent in doc_q.sents]
        #print('[question]number of sentences: ', len(sentences_q))
        out = []
        #use 8 passages
        ns = len(sentences_q)//8 #8 passages, ns sentences each passage
        if ns==0:
            ns = 1
        for j in range(8):
            p = ' '.join(s for s in sentences_q[ns*j:ns*(j+1)])
            if j==7:
                p += ' '.join(s for s in sentences_q[ns*(j+1):])
            #print(j, p)
            #out.append(use_embed([p])[0])
            out.append(p)
        #out = np.array(out).T
        qss.append(out)
        ##for answer text
        doc_a = nlp(a)
        sentences_a = [sent.string.strip() for sent in doc_a.sents]
        #print('[answer]number of sentences: ', len(sentences_a))
        out = []
        #use 8 passages
        ns = len(sentences_a)//8 #8 passages, ns sentences each passage
        if ns==0:
            ns = 1
        for j in range(8):
            p = ' '.join(s for s in sentences_a[ns*j:ns*(j+1)])
            if j==7:
                p += ' '.join(s for s in sentences_a[ns*(j+1):])
            #print(j, p)
            #out.append(use_embed([p])[0])
            out.append(p)
        #out = np.array(out).T
        ass.append(out)
        
        ##as a whole sentence
        #qss.append([t+' '+q]) 
        #ass.append([a])
    
    output = [np.asarray([special_char_q, special_char_a, cap_char_q, cap_char_a, unique_words_q, unique_words_a], dtype=np.float32).T, 
              np.asarray(category_index, dtype=np.int32),
              np.asarray(host_index, dtype=np.int32)
             ]
    
    qss = np.asarray(qss, dtype=str)
    ass = np.asarray(ass, dtype=str)
    for i in range(8):
        output.append(qss[:,i:i+1])
    for i in range(8):
        output.append(ass[:,i:i+1])
    return output

def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

In [10]:
# np.asarray(qss, dtype=str).shape

In [11]:
# use_embed(np.asarray(qss, dtype=str)[0]).shape

#### 3. Create model

`compute_spearmanr()` is used to compute the competition metric for the validation set
<br><br>
`CustomCallback()` is a class which inherits from `tf.keras.callbacks.Callback` and will compute and append validation score and validation/test predictions respectively, after each epoch.
<br><br>
`bert_model()` contains the actual architecture that will be used to finetune BERT to our dataset. It's simple, just taking the sequence_output of the bert_layer and pass it to an AveragePooling layer and finally to an output layer of 30 units (30 classes that we have to predict)
<br><br>
`train_and_predict()` this function will be run to train and obtain predictions

In [14]:
#e = tf.ones((4, 512, 768))

#[e,e]

#tf.keras.layers.Concatenate(axis=-1)([e,e]).shape

#tf.reduce_sum([e,e], axis=0).shape
#tf.maximum(e,e).shape

In [26]:
def compute_spearmanr_ignore_nan(trues, preds):
    rhos = []
    for tcol, pcol in zip(np.transpose(trues), np.transpose(preds)):
        rhos.append(spearmanr(tcol, pcol).correlation)
    return np.nanmean(rhos)


class CustomCallback(tf.keras.callbacks.Callback):
    
    def __init__(self, valid_data, test_data=None, batch_size=16, fold=None, stage2=False):

        self.valid_inputs = valid_data[0]
        self.valid_outputs = valid_data[1]
        self.test_inputs = test_data
        
        self.batch_size = batch_size
        self.fold = fold
        self.stage2 = stage2
        
    def on_train_begin(self, logs={}):
        self.valid_predictions = []
        self.test_predictions = []
        
    def on_epoch_end(self, epoch, logs={}):
        self.valid_predictions.append(
            self.model.predict(self.valid_inputs, batch_size=self.batch_size))
        
        rho_val = compute_spearmanr_ignore_nan(
            self.valid_outputs, np.average(self.valid_predictions, axis=0))
        
        print("\nvalidation rho: %.4f" % rho_val)
        
        if self.fold is not None and epoch>1:
            if self.stage2:
                self.model.save_weights(CKPT_SAVE_PATH+f'fold{fold}-epoch{epoch}-stage2.h5py')
            else:
                self.model.save_weights(CKPT_SAVE_PATH+f'fold{fold}-epoch{epoch}.h5py')
        
#         self.test_predictions.append(
#             self.model.predict(self.test_inputs, batch_size=self.batch_size)
#         )

def UniversalEmbedding(x):
    results = use_embed(tf.cast(tf.squeeze(x), tf.string))
    #print(results)
    return tf.keras.backend.concatenate([results])

def use_model():
    
#     qss = tf.keras.layers.Input((8,), dtype=tf.string)
#     ass = tf.keras.layers.Input((8,), dtype=tf.string)
#     qss_embed = use_embed(qss)
#     ass_embed = use_embed(ass)
    
    qss = [tf.keras.layers.Input((1,), dtype=tf.string) for _ in range(8)]
    ass = [tf.keras.layers.Input((1,), dtype=tf.string) for _ in range(8)]
    #qss_embed_layer = tf.keras.layers.Lambda(UniversalEmbedding, output_shape=(1,))
    #ass_embed_layer = tf.keras.layers.Lambda(UniversalEmbedding, output_shape=(1,))
    use_embed_layer = tf.keras.layers.Lambda(UniversalEmbedding, output_shape=(1,))
    qss_embed = [tf.keras.backend.expand_dims(use_embed_layer(_qss)) for _qss in qss]#[(N, 512, 1),...,]
    ass_embed = [tf.keras.backend.expand_dims(use_embed_layer(_ass)) for _ass in ass]
    qss_embed = tf.keras.layers.Concatenate()(qss_embed)#(N, 512, 8)
    ass_embed = tf.keras.layers.Concatenate()(ass_embed)
    
    feats = tf.keras.layers.Input((6,), dtype=tf.float32)#set dim of additional numeric feats
    category_index = tf.keras.layers.Input((1,), dtype=tf.int32)
    host_index = tf.keras.layers.Input((1,), dtype=tf.int32)
    
    #q_embedding = tf.keras.layers.SpatialDropout1D(0.2)(q_embedding)
    #a_embedding = tf.keras.layers.SpatialDropout1D(0.2)(a_embedding)
    
    qa_embed = tf.keras.layers.Concatenate()([qss_embed, ass_embed])
    qa1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True, 
                                                            activation='tanh', recurrent_activation='sigmoid', 
                                                            recurrent_dropout=0, unroll=False, use_bias=False
                                                            ))(qa_embed)
    qa2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True, 
                                                            activation='tanh', recurrent_activation='sigmoid', 
                                                            recurrent_dropout=0, unroll=False, use_bias=False
                                                            ))(qa1)
    
    qa1_avgpool = tf.keras.layers.GlobalAveragePooling1D()(qa1)
    qa2_avgpool = tf.keras.layers.GlobalAveragePooling1D()(qa2)
    #qa_maxpool = tf.keras.layers.GlobalMaxPool1D()(qa)
    q_avgpool = tf.keras.layers.GlobalAveragePooling1D()(qss_embed)
    a_avgpool = tf.keras.layers.GlobalAveragePooling1D()(ass_embed)
    
    category_embed_layer = tf.keras.layers.Embedding(6, 8, input_length=1)
    host_embed_layer = tf.keras.layers.Embedding(64, 8, input_length=1)
    cat_embed = category_embed_layer(category_index)
    host_embed = host_embed_layer(host_index)
    cat_embed = tf.keras.layers.Flatten()(cat_embed)
    host_embed = tf.keras.layers.Flatten()(host_embed)
    
    #x = tf.keras.layers.Concatenate()([q, a, feats, cat_embed, host_embed])
    x = tf.keras.layers.Concatenate()([qa1_avgpool, qa2_avgpool, q_avgpool, a_avgpool,
                                       feats, cat_embed, host_embed])
    
    x = tf.keras.layers.Dropout(0.2)(x)
    
    x = tf.keras.layers.Dense(30, activation='sigmoid')(x)

    model = tf.keras.models.Model(inputs=[feats, category_index, host_index]+qss+ass, outputs=x)
    
    return model


def train_and_predict(model, train_data, valid_data, test_data, 
                      learning_rate, epochs, batch_size, loss_function, fold, stage2=False):
    """
    multipliers = {'dense_1': 0.5, 'dense_2': 0.4}
    optimizer = LearningRateMultiplier(tf.keras.optimizers.Adam, 
                                        lr_multiplier=multipliers, learning_rate=learning_rate)
    
    """
    custom_callback = CustomCallback(
        valid_data=(valid_data[0], valid_data[1]), 
        test_data=test_data,
        batch_size=batch_size,
        fold=fold, stage2=stage2)

    #decay_steps = 1000
    #lr_decayed_fn = tf.keras.experimental.CosineDecay(learning_rate, decay_steps)
    #clr = CyclicLR(base_lr=2e-5, max_lr=5e-5, step_size=2000., mode='triangular')

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)#clipvalue=0.5
    model.compile(loss=loss_function, optimizer=optimizer)
    model.fit(train_data[0], train_data[1], epochs=epochs, 
              batch_size=batch_size, callbacks=[custom_callback])
    
    return custom_callback

def predict(model, test_data, load_weights_path):
    model.load_weights(load_weights_path)
    return model.predict(test_data, batch_size=BATCH_SIZE)

In [27]:
model = use_model()
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_76 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_77 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_78 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_79 (InputLayer)           [(None, 1)]          0                                            
____________________________________________________________________________________________

In [28]:
#inputs[10].shape, outputs.shape

#### 4. Obtain inputs and targets, as well as the indices of the train/validation splits

In [22]:
if TRAIN_PREDICT == 'train':
    outputs = compute_output_arrays(df_train, output_categories)
    inputs = compute_input_arrays(df_train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

In [23]:
import pickle

if TRAIN_PREDICT == 'train':
    if not os.path.isfile('../data/gkf%d.pkl'%SEED):
        print('Create gkf')
        gkf = GroupKFold(n_splits=5).split(X=df_train.question_body, groups=df_train.question_body)
        with open('../data/gkf%d.pkl'%SEED, 'wb') as f:
            pickle.dump(list(gkf), f)
    else:
        print('Load gkf')
        with open('../data/gkf%d.pkl'%SEED, 'rb') as f:
            gkf = pickle.load(f)

Load gkf


In [14]:
#print(len(inputs), inputs[6].shape, inputs[7], inputs[8])
#inputs[6][:,4:]
# for i,(train_idx,val_idx) in enumerate(gkf):
#     if i==0:
#         break
# train_idx[20:30]
#inputs[8][0].shape

#gkf0 = list(gkf)

#### 5. Training, validation and testing

Loops over the folds in gkf and trains each fold for 5 epochs --- with a learning rate of 1e-5 and batch_size of 8. A simple binary crossentropy is used as the objective-/loss-function. 

In [24]:
NUM_EPOCHS = 4
BATCH_SIZE = 8
LearningRate = 3e-5

In [25]:
def custom_bce_loss(y_true, y_pred):
    """
    nunique = [df_train[col].nunique() for col in output_categories]#count unique values of each column
    weights_dict = {5:0.5, 9:1.0, 17:1.5, 3:0.5}
    weights = [weights_dict[i] for i in nunique]
    """
    weights = tf.convert_to_tensor([1. , 1. , 0.5, 0.5, 0.5, 0.5, 1. , 1. , 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. , 1. , 1. , 1. , 1. , 1.5,
       0.5, 0.5, 0.5, 1. ], dtype=float)
    weights = weights/K.mean(weights)
    loss = tf.multiply(y_true, K.log(y_pred+K.epsilon())) + tf.multiply((1-y_true), K.log(1-y_pred+K.epsilon()))
    loss = tf.multiply(loss, weights)
    bce_loss = tf.reduce_mean(-loss)
    return bce_loss
#     y_true_clip = K.clip(y_true, K.epsilon(), 1)
#     y_pred_clip = K.clip(y_pred, K.epsilon(), 1)
#     kl_loss = tf.reduce_mean(tf.reduce_sum(y_true_clip * K.log(y_true_clip / y_pred_clip), axis=0))
#     return bce_loss*0.9 + kl_loss*0.1

#y_pred = tf.random.uniform((8, 30))
#y_true = tf.random.uniform((8, 30))

#custom_bce_loss(y_true, y_pred)

In [26]:
## training
## if stage2, modify 2 places
if TRAIN_PREDICT == 'train':
    histories = []
    for fold, (train_idx, valid_idx) in enumerate(gkf):

        # will actually only do 1 fold (out of 5) to compare models
        if fold ==0:
            print('========training fold %d========'%fold)
            K.clear_session()
            
            model = bert_model()
            
            #prepare dataset
            train_inputs = [inputs[i][train_idx] for i in range(len(inputs))]
            train_outputs = outputs[train_idx]
            valid_inputs = [inputs[i][valid_idx] for i in range(len(inputs))]
            valid_outputs = outputs[valid_idx]

            # history contains two lists of valid and test preds respectively:
            #  [valid_predictions_{fold}, test_predictions_{fold}]
            history = train_and_predict(model, 
                              train_data=(train_inputs, train_outputs), 
                              valid_data=(valid_inputs, valid_outputs),
                              test_data=None, 
                              learning_rate=LearningRate, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE,
                              loss_function=custom_bce_loss, fold=fold, 
                                        stage2=False)#'binary_crossentropy'
            histories.append(history)

# ## training full trainset to lift LB score in the final
# if TRAIN_PREDICT == 'train':
#     histories = []
#     print('========Start training========')
#     K.clear_session()
#     model = bert_model()

#     train_inputs = inputs
#     train_outputs = outputs

#     history = train_and_predict(model, 
#                       train_data=(train_inputs, train_outputs), 
#                       valid_data=None,
#                       test_data=None, 
#                       learning_rate=LearningRate, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE,
#                       loss_function='binary_crossentropy', fold=fold)

#     histories.append(history)

========training fold 0========
Train on 4863 samples
Epoch 1/4


4856/4863 [============================>.] - ETA: 3s - loss: 0.4054
validation rho: 0.3786
4863/4863 [==============================] - 2687s 553ms/sample - loss: 0.4054
Epoch 2/4
4856/4863 [============================>.] - ETA: 2s - loss: 0.3812
validation rho: 0.3994
4863/4863 [==============================] - 2216s 456ms/sample - loss: 0.3812
Epoch 3/4
4856/4863 [============================>.] - ETA: 2s - loss: 0.3688
validation rho: 0.4065
4863/4863 [==============================] - 2225s 457ms/sample - loss: 0.3688
Epoch 4/4
4856/4863 [============================>.] - ETA: 2s - loss: 0.3573
validation rho: 0.4081
4863/4863 [==============================] - 2219s 456ms/sample - loss: 0.3573


In [ ]:
#more ideas todo: 
#1.1 modify model
    #sliding window for sentences --> USE embed --> LSTM
    #https://github.com/google-research/bert/issues/66
    #https://www.kaggle.com/c/google-quest-challenge/discussion/120185#689854
#post-process, transform prediction to rank, to get averaged rank across folds, then assign new predictions
#1.2 OOV words spelling correction
#1.3 train >4 epochs
#2. loss, ranking loss?
#3. add custom new tokens?(e.g stackoverflow)
#4. roberta?alxnet?
#5. RankGauss average folds?
#6. freeze some layers of bert?
"""
CV history
---------------
#### bert-base ####
Epoch 5/15
4856/4863 [============================>.] - ETA: 0s - loss: 0.3091
validation rho: 0.3923
4863/4863 [==============================] - 339s 70ms/sample - loss: 0.3091
------LB=0.346

switch to HuggingFace
----------------------
SEEMS DEPENDS ON THE SEED!!!
----------------------------
t + q[:1/2], q[1/2:], a
same

category + host + t + q, category + host + a
same


CUSTOM LOSS
Epoch 4/4
4856/4863 [============================>.] - ETA: 0s - loss: 0.3612
validation rho: 0.3989
4863/4863 [==============================] - 711s 146ms/sample - loss: 0.3612

Epoch 4/4
4856/4863 [============================>.] - ETA: 0s - loss: 0.3597
validation rho: 0.3998
4863/4863 [==============================] - 724s 149ms/sample - loss: 0.3597

add 2 feats
Epoch 4/4
4856/4863 [============================>.] - ETA: 0s - loss: 0.3616
validation rho: 0.4049
4863/4863 [==============================] - 718s 148ms/sample - loss: 0.3616

4 feats
Epoch 4/4
4856/4863 [============================>.] - ETA: 0s - loss: 0.3592
validation rho: 0.4027
4863/4863 [==============================] - 706s 145ms/sample - loss: 0.3591

add cat+host embed dim=16
Epoch 4/4
4856/4863 [============================>.] - ETA: 0s - loss: 0.3594
validation rho: 0.4005
4863/4863 [==============================] - 698s 144ms/sample - loss: 0.3594

embed dim=8
Epoch 4/4
4856/4863 [============================>.] - ETA: 0s - loss: 0.3592
validation rho: 0.4071
4863/4863 [==============================] - 734s 151ms/sample - loss: 0.3592

add LSTM features
validation rho: 0.4081 --fold0
validation rho: 0.4050 --fold1
validation rho: 0.4138 --fold2
validation rho: 0.4018 --fold3
validation rho: 0.3982 --fold4
LB=0.388, CV=0.405

CV 0.397 to 0.405, LB 0.387 to 0.388 ???


add t+[SPECIAL TOKEN]+q
validation rho: 0.4113 --fold0
validation rho: 0.4014 --fold1
validation rho: 0.4224 --fold2
validation rho: 0.4055 --fold3
validation rho: 0.3961 --fold4
CV=0.407, LB=0.387

--------
epochs progression:

4856/4863 [============================>.] - ETA: 2s - loss: 0.4053
validation rho: 0.3832
4863/4863 [==============================] - 2045s 421ms/sample - loss: 0.4052
Epoch 2/4
4856/4863 [============================>.] - ETA: 2s - loss: 0.3799
validation rho: 0.4016
4863/4863 [==============================] - 1995s 410ms/sample - loss: 0.3799
Epoch 3/4
4856/4863 [============================>.] - ETA: 2s - loss: 0.3680
validation rho: 0.4092
4863/4863 [==============================] - 2002s 412ms/sample - loss: 0.3680
Epoch 4/4
4856/4863 [============================>.] - ETA: 2s - loss: 0.3557
validation rho: 0.4108
4863/4863 [==============================] - 1998s 411ms/sample - loss: 0.3557

"""

#### 6. Process and submit test predictions

First the test predictions are read from the list of lists of `histories`. Then each test prediction list (in lists) is averaged. Then a mean of the averages is computed to get a single prediction for each data point. Finally, this is saved to `submission.csv`

In [12]:
if TRAIN_PREDICT == 'predict':
    
    test_inputs = compute_input_arrays(df_test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
    
    model = bert_model()
    
    test_predictions = [predict(model, test_inputs, load_weights_path=ckpt_load_path) 
                        for ckpt_load_path in CKPT_LOAD_PATH]
    #test_predictions = [np.average(test_predictions[i], axis=0) for i in range(len(test_predictions))]
    test_predictions = np.mean(test_predictions, axis=0)

    df_sub.iloc[:, 1:] = test_predictions

    df_sub.to_csv('submission.csv', index=False)

NameError: name 'compute_input_arays' is not defined

In [ ]:
df_sub.head(10)

In [ ]:
# ##post process
# cols2process = df_train.columns.tolist()[11:]
# print(len(cols2process))

In [ ]:
# for col in tqdm(cols2process):
#     ###step1
#     quantiles = df_train[col].value_counts()/len(df_train.index)
#     quantiles = quantiles.to_dict()
#     quantiles = {k: v for k, v in sorted(quantiles.items(), key=lambda item: item[0])}
#     ks = list(quantiles.keys())
#     vs = list(quantiles.values())
#     qs = np.cumsum(vs)
#     #print(ks)
#     #print(qs)
#     ###step2
#     qs = np.quantile(df_sub[col], qs)
#     #print(qs)
#     for i in range(len(qs)):
#         if i==0:
#             q = qs[0]
#             df_sub.loc[df_sub[col]<q, col] = ks[i]
#         elif i>0 and i<=len(qs)-1:
#             q0,q1 = qs[i-1], qs[i]
#             df_sub.loc[(df_sub[col]<q1)&(df_sub[col]>=q0), col] = ks[i]

In [ ]:
# df_sub.head(10)

In [ ]:
# df_sub.to_csv('submission.csv', index=False)

## maybe post-processing

In [12]:
model = bert_model()

In [18]:
for fold, (train_idx, valid_idx) in enumerate(gkf):
    #prepare dataset
    train_inputs = [inputs[i][train_idx] for i in range(len(inputs))]
    train_outputs = outputs[train_idx]
    valid_inputs = [inputs[i][valid_idx] for i in range(len(inputs))]
    valid_outputs = outputs[valid_idx]

#     model.load_weights('../checkpoint/bert-base-uncased-v3/fold0-epoch3.h5py')
#     valid_predictions0 = model.predict(valid_inputs, batch_size=BATCH_SIZE)
#     rho_val = compute_spearmanr_ignore_nan(valid_outputs, valid_predictions0)
#     print('rho_val: ', rho_val)
    
    model.load_weights('../checkpoint/bert-base-uncased-v4/fold0-epoch3.h5py')
    valid_predictions1 = model.predict(valid_inputs, batch_size=BATCH_SIZE)
    rho_val = compute_spearmanr_ignore_nan(valid_outputs, valid_predictions1)
    print('rho_val: ', rho_val)
    
    if fold==0:
        break

rho_val:  0.39507176135825195


In [45]:
preds = [valid_predictions, valid_predictions1]

In [139]:
from scipy.stats import rankdata

cols = range(30)#[0]

def rank_average(cols):
    preds_avg = np.empty_like(preds[0])
    for col in cols:
        avg_rank = np.mean([rankdata(p[:,col], method='dense') for p in preds], axis=0).round(0).astype(int)
        avg_rank = rankdata(avg_rank, method='dense')-1
        nunique = np.unique(avg_rank).shape[0]
        arrays = np.array([p[:,col] for p in preds])
        ranges = arrays.min(), arrays.max()
        uniform = np.linspace(ranges[0],ranges[1],nunique)
        refined = np.array([uniform[i] for i in avg_rank])
        preds_avg[:,col] = refined
    return preds_avg

In [140]:
refined = rank_average(cols)

In [142]:
rho_val = compute_spearmanr_ignore_nan(valid_outputs, refined)
rho_val

0.4036110465409228